In [1]:
import os
from os.path import join as pjoin
import pdb
import json

import numpy as np

import plotly.graph_objs as go
import plotly as py
import ipywidgets as widgets
py.offline.init_notebook_mode(connected=True)

In [13]:
os.system(f"make iftExtractFalsePositiveSVoxelStats -C $NEWIFT_DIR/demo/BrainAsymmetryAnalysis/TBME/exp02_understanding_false_positives")

0

In [14]:
work_dir = "/Users/hisamuka/workspace/exps/2019_Martins_TBME_AbnormalAsymmetryDetection"
saad_result_dir = os.path.join(work_dir, "exps/01_why_good_supervoxels/SAAD_ISBI/split_01")
img_dir = os.path.join(work_dir, "bases/ATLAS-304/3T/regs/nonrigid/final")
normal_asym_map = os.path.join(work_dir, "exps/normal_asym_map.nii.gz")
gt_dir = os.path.join(work_dir, "bases/ATLAS-304/3T/regs/nonrigid/labels/primary_stroke")

In [16]:
exp_dir = os.path.join(work_dir, "exps/02_understanding_false_positives")

right_hem_mask_path = os.path.join(work_dir, "template/mni152_nonlinear_sym_right_hemisphere_close.nii.gz")
false_positive_stats_json = os.path.join(exp_dir, "svoxels_stats.json")
log_file = os.path.join(exp_dir, "log.txt")

os.system(f"iftExtractFalsePositiveSVoxelStats --saad-result-dir {saad_result_dir} --image-dir {img_dir} --ground-truth-dir {gt_dir} --right-brain-mask {right_hem_mask_path} --output-json {false_positive_stats_json} --normal-asymmetry-map {normal_asym_map} > {log_file}")

stats = {}
with open(false_positive_stats_json, 'r') as f:
        stats = json.load(f)

print("Done!!!")

Done!!!


In [29]:
min_svoxel_volume = min(stats["svoxels-volume"])
max_svoxel_volume = max(stats["svoxels-volume"])
min_edt = min(stats["edt"])
max_edt = max(stats["edt"])
min_mean_asym = min(stats["mean-asym"])
max_mean_asym = max(stats["mean-asym"])


data = [
    go.Parcoords(
        line = dict(color = '#ee6e73'),
        dimensions = list([
            dict(
                label = 'Dist. Centroid to the Border (mm)',
                values = stats["edt"],
                tickformat = ".2f"),
            dict(
                label = 'Supervoxel Volume (mm3)',
                values = stats["svoxels-volume"],
                tickvals = np.arange(min_svoxel_volume, max_svoxel_volume + 1, (max_svoxel_volume - min_svoxel_volume) / 20),
                tickformat = ".2f"),
            dict(
                label = 'Mean Asymmetry inside SVoxel',
                values = stats["mean-asym"],
                tickformat = ".2f"),
        ])
    )
]

layout = go.Layout(
    title = "False Positive Supervoxels"
)

fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)